In [1]:
import json
!pip install jsonlines
!pip install json_lines

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import json
f = open("drive/My Drive/Colab Notebooks/real_question.json",)
data = json.load(f)

print(len(data))
f.close()

Mounted at /content/drive
99991


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(['go', 'kind', 'sort', 'think','see','still','question','expect','guy','year','quarter','end','time','guess','sense','way','thank','give_us','thanks','steve','number','point','comment','business','range','thought','group','mix','basis','little_bit','company','month','thing','couple','color','term','side','update','issue','talk','follow','guidance','segment','assumption','care','level','part','line','admission','good_morning'])

In [7]:
# Convert to list

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data[:1000]))

print(data_words[0])

['thanks', 'very', 'much', 'operator', 'hi', 'everybody', 'thank', 'you', 'very', 'much', 'for', 'your', 'time', 'just', 'on', 'the', 'bookings', 'over', 'the', 'past', 'mean', 'this', 'is', 'like', 'very', 'short', 'term', 'focused', 'but', 'are', 'you', 'hearing', 'from', 'any', 'of', 'your', 'corporate', 'accounts', 'about', 'adjusting', 'bookings', 'or', 'whatever', 'for', 'the', 'given', 'united', 'performance', 'over', 'the', 'last', 'couple', 'of', 'weeks', 'with', 'various', 'incidences']


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['thanks', 'very', 'much', 'operator', 'hi_everybody', 'thank', 'you', 'very', 'much', 'for', 'your', 'time', 'just', 'on', 'the', 'bookings', 'over', 'the', 'past', 'mean', 'this', 'is', 'like', 'very', 'short_term', 'focused', 'but', 'are', 'you', 'hearing', 'from', 'any', 'of', 'your', 'corporate', 'accounts', 'about', 'adjusting', 'bookings', 'or', 'whatever', 'for', 'the', 'given', 'united', 'performance', 'over', 'the', 'last', 'couple', 'of', 'weeks', 'with', 'various', 'incidences']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    #return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    result = []
    for doc in texts:
      new_doc = []
      for word in doc:
        if word not in stop_words:
          new_doc.append(word)
      result.append(new_doc)
    return result

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

#allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
#allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN','PROPN'])

data_lemmatized = remove_stopwords(data_lemmatized)
print(data_lemmatized[0])

['operator', 'hi_everybody', 'booking', 'account', 'booking', 'united', 'performance', 'week', 'incidence']


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4      0.083314 -0.197599       1        1  8.053663
9     -0.074611  0.171769       2        1  7.739448
3     -0.298678 -0.150330       3        1  7.266467
11    -0.123789  0.200473       4        1  7.135271
1      0.120660  0.021660       5        1  6.546819
15    -0.028068 -0.025676       6        1  5.488628
6     -0.134943 -0.077749       7        1  5.440268
2      0.044182 -0.100242       8        1  4.847584
13    -0.040933  0.050439       9        1  4.750198
18     0.095558  0.008833      10        1  4.668245
10     0.094028  0.037308      11        1  4.578208
17     0.075266  0.058225      12        1  4.307000
19     0.082020  0.014505      13        1  3.969776
7      0.023803  0.000067      14        1  3.924235
8     -0.045806  0.104076      15        1  3.886850
16     0.031609 -0.139764      16        1  3.719831
12     0.061879 -0.007944      17        1  3.705875
5     -0.018974  0.033199      18        1  3.389904
0     -0.005737 -0.035676      19        1  3.340455
14     0.059219  0.034427      20        1  3.241274, topic_info=            Term        Freq       Total Category  logprob  loglift
59          rate  193.000000  193.000000  Default  30.0000  30.0000
73        growth  185.000000  185.000000  Default  29.0000  29.0000
81        margin  123.000000  123.000000  Default  28.0000  28.0000
82        market  143.000000  143.000000  Default  27.0000  27.0000
58      pressure   76.000000   76.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
421   commercial    1.754170    2.616674  Topic20  -5.2633   3.0293
255         area   10.403064   17.430741  Topic20  -3.4831   2.9131
279   commentary    8.380877   15.504670  Topic20  -3.6993   2.8140
672      product    3.448846   17.073784  Topic20  -4.5872   1.8297
238  acquisition    2.570054   23.827874  Topic20  -4.8813   1.2023

[727 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1748     20  0.863514           aar
317       2  0.951613       ability
840       5  0.860891           aca
687      12  0.907511  acceleration
424       9  0.927928        access
...     ...       ...           ...
1614      8  0.882816          wrap
1340     10  0.909270            xl
947       1  0.920127     yesterday
713      18  0.910089         yield
1729      2  0.843425          york

[743 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 4, 12, 2, 16, 7, 3, 14, 19, 11, 18, 20, 8, 9, 17, 13, 6, 1, 15])

In [ ]:
tweet_content = []
json_new = json_content_eng[700:800]
for dic in json_new:
    text = dic['full_text']
    tweet_content.append(text)
with open("drive/My Drive/tweet_content.json", "w") as write_file:
    json.dump(tweet_content, write_file, indent=4)